In [16]:
# generate PQ

def generate_pq (G, gens):
    pq = set()
    queue = [g for g in gens]
    
    N = len(G)
    
    rs = G.rewriting_system()
    rs.make_confluent()
    
    while len(queue) > 0:
        gpre = queue.pop()
        
        g = rs.reduce(gpre)
        
        if g in pq:
            continue
        
        for x in pq:
            queue.append(x*g*(x^(-1)))
            queue.append(g*x*(g^(-1)))
        
        for n in range(N):
            queue.append(g^n)
        
        pq.add(g)
        
    return pq

In [40]:
G = groups.presentation.Symmetric(5)

In [50]:
G

Finitely presented group < a, b | b^2, a^3*(b*a^-1)^3*b*a, (b*a^2*b*a^-2)^2, (b*a*b*a^-1)^3 >

In [41]:
G.gens()

(a, b)

In [42]:
pq = generate_pq(G, G.gens()); pq

{1,
 b,
 b*a^-1*b*a*b,
 a^-1*b*a,
 (a^2*b)^2*a*b*a^-1,
 a*b*a^-1,
 b*a*b*a^-1*b,
 a^2*b*a^-2,
 a*b*a^2*b*a^-1,
 a^-2*b*a*b*a^-1,
 a,
 a^-1*(a^-1*b)^2,
 b*a*b,
 b*a^-2*b*a^-1,
 b*a^-1*b,
 a^-1,
 a^-2*b*a^2,
 a^2*b*a^-1*b*a,
 a*(a*b)^2,
 a^-1*b*a^-2*b*a,
 b*a^2*b*a,
 a*b*a^-1*b*a^2,
 a*b*a^2*b,
 (a*b*a)^2*b,
 a^-1*b*a^2*b*a,
 b*a^-2*b,
 (a*b)^2*a^-1,
 a^2,
 a*b*a^-2*b*a^-1,
 (a^-1*b)^2*a,
 (a^-1*b*a^-1)^2*b,
 a^-1*b*a*b*a^-2,
 a^-2,
 a^-1*b*a^-2*b,
 b*a^2*b}

In [43]:
len(pq)

35

In [44]:
len(G)

120

In [45]:

enumed = list(enumerate(pq))
#gens = list(G.gens())

rs = G.rewriting_system()
rs.make_confluent()

groupIndex = dict([(rs.reduce(g), i) for i, g in enumed])

def indexOf(g):
    g1 = rs.reduce(g)
    return groupIndex[g1]
    #for i, g1 in enumed:
        #if g == g1:
            #return i

def forOneG(t):
    (i, g) = t
    grelations = []
    
    indexInv = indexOf(g^(-1))
    
    for n in range(len(G)):
        gn = g^n
        if gn == g and n > 1:
            break
        indexN = indexOf(gn)
        grelations.append([indexInv + 1]*n + [indexN + 1])

    for i1, g1 in enumed:
        #i1 = indexOf(g1)
        indexInv1 = indexOf(g1^(-1))
        indexInv2 = indexOf((g1*g*g1^(-1))^(-1)) 
        grelations.append([i1 + 1, i + 1, indexInv1 + 1, indexInv2 + 1])

    return grelations

relations = []

#for i, g in enumed:
    #rel = forOneG(i, g)
    #relations.extend(rel)

from multiprocessing import Pool

pool = Pool(processes = 7)
result = pool.map(forOneG, enumed)
relations = [rel for rels in result for rel in rels]
pool.close()
pool.join()

In [46]:
FreeG = FreeGroup(len(pq))

In [47]:
FG = FreeG / relations

In [48]:
FGS = FG.simplified()
FGS

Finitely presented group < x7, x30 | x30^2, x7^5, (x30*x7^-1)^4, (x30*x7^2*x30*x7^-2)^2, (x7^-1*x30*x7*x30)^3 >

In [49]:
FGS.structure_description()

'S5'